In [1]:
from langchain_core.output_parsers import PydanticOutputParser

email_conversation = """From: 테디 (teddy@teddynote.com)
To: 이은채 대리님 (eunchae@teddyinternational.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 이은채 대리님,

저는 테디노트의 테디입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.

저희 테디노트의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 뵐 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 이은채 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

테디
테디노트 AI 솔루션팀"""

In [27]:
from pydantic import BaseModel, Field

# 이메일 본문으로부터 주요 엔티티 추출
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company: str = Field(description="메일을 보낸 사람의 회사 정보")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [4]:
## LCEL 구조

# chain = prompt | llm | output_parser


In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [8]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response
import os

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model=os.getenv("BASE_MODEL"),
    temperature=0
)

answer = llm.stream("스위스이 수도가 어디야")
stream_response(answer)

스위스의 수도는 **베른(Bern)**입니다. 스위스 연방의 행정 중심지로, 아름다운 중세 도시로 유명하죠! 추가 질문 있으신가요?

In [28]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=EmailSummary)
output_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "company": {"description": "메일을 보낸 사람의 회사 정보", "title": "Company", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "company", "

In [14]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are a helpful assistant. Please answer the following questions in KOREAN.

    #QUESTION:
    다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

    #EMAIL CONVERSATION:
    {email_conversation}

    #FORMAT:
    {format}
    """
)

prompt = prompt.partial(format=output_parser.get_format_instructions())

In [29]:
# 체인 생성
chain = prompt | llm |output_parser

In [30]:
# 체인 실행
answer = chain.invoke({"email_conversation": email_conversation})

In [18]:
answer.person

'테디'

In [31]:
answer.company

'테디노트'

In [33]:
query = f"{answer.person} {answer.company} {answer.email}"
query

'테디 테디노트 teddy@teddynote.com'

In [22]:
print(answer)

person='테디' phone_number='' company='테디노트' email='teddy@teddynote.com' subject='RAG 솔루션 시연 관련 미팅 제안' summary='테디노트의 테디가 이은채 대리에게 AI 및 RAG 솔루션의 경험을 소개하며, 귀사의 데이터 활용과 비즈니스 의사결정을 지원할 수 있음을 강조. RAG 솔루션의 기능과 적용 방안을 시연하기 위한 미팅을 제안하며, 다음 주 목요일(7월 18일) 오전 10시 귀사 사무실에서 만나자고 함. 다른 일정이 있으면 조율 가능.' date='다음 주 목요일(7월 18일) 오전 10시'


## 검색: SERP API

참고: https://serpapi.com/integrations/python

In [23]:
import os

os.environ["SERPAPI_API_KEY"] = "e03eb1a016c7c1d677d74fe7a4a6c5afb2972b4fb17421efaebf68b128249499"

In [24]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

search = SerpAPIWrapper(params=params)

In [34]:
search.run("테디노트 site:naver.com")

'[\'패스트캠퍼스에서 진행 중인 테디노트의 RAG 비법노트를 덮썩 물었다. 코드잇이나 다른 교육 강좌에는 없어서 메리트가 있었다. \\u200b. \\u200b. 처음부터 패스트 ...\', "... 테디노트 (teddylee777.github.io)운영(\'17년~현재) 전 - 삼성전자SCSA1기 - 삼성전자무선사업부삼성노트,테마,갤러리 앱개발 - C-LAB사내벤처독립및 ...", \'국내에선 테디노트가 이 영역의 영웅이나 다름 없는데, 비록 나는 테디노트로 공부를 하진 않았지만 그의 노력과 설명력을 여기서 꼭 칭송하고 싶다.\']'

In [35]:
search.run(query)

'[\'테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 (Windows). LocalModels. GGUF · HuggingFace gguf 파일을 Ollama 로딩 · TeddyNote.\', \'문서 관리를 위한 메타데이터 태깅은 필수적이지만 번거로울 수 있습니다. OpenAI 기반의 자동화된 메타데이터 태깅 방법을 통해 이 과정을 효율적으로 만드는 방법을 알아 ...\', \'데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 \\u200d♂️ ...more 데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 ...\']'

In [36]:
search_result = search.run(query)

In [38]:
search_result = eval(search_result)

In [39]:
type(search_result)

list

In [40]:
search_result[0]

'테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 (Windows). LocalModels. GGUF · HuggingFace gguf 파일을 Ollama 로딩 · TeddyNote.'